In [20]:
import glob
import skimage.io as io
import numpy as np
#import commonfunctions as cf
import sklearn
from skimage.feature import hog
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import pickle
import cv2
import random
import os
from PIL import Image, ImageEnhance
from PIL import Image, ImageEnhance
import cv2
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict, RandomizedSearchCV
from sklearn.metrics import *
from sklearn.decomposition import PCA
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier

In [21]:
'''
if you want to load the data again to work with them
'''

male_data=np.load('male_data_small.npy',allow_pickle=True)
female_data=np.load('female_data_small.npy',allow_pickle=True)

In [22]:
def get_contour_pixels(bw_image):
    contours, _= cv2.findContours(
        bw_image, cv2.RETR_TREE, 
        cv2.CHAIN_APPROX_NONE
        ) 
    # contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[1:]
    
    img2 = bw_image.copy()[:,:,np.newaxis]
    img2 = np.concatenate([img2, img2, img2], axis = 2)
    return contours

In [23]:
N_ANGLE_BINS = 40
BIN_SIZE = 360 // N_ANGLE_BINS
LEG_LENGTH = 25

def get_hinge_features(bw_image,contours):
    #contours = get_contour_pixels(bw_image)
    
    hist = np.zeros((N_ANGLE_BINS, N_ANGLE_BINS))
        
    # print([len(cnt) for cnt in contours])
    for cnt in contours:
        n_pixels = len(cnt)
        if n_pixels <= LEG_LENGTH:
            continue
        
        points = np.array([point[0] for point in cnt])
        xs, ys = points[:, 0], points[:, 1]
        point_1s = np.array([cnt[(i + LEG_LENGTH) % n_pixels][0] for i in range(n_pixels)])
        point_2s = np.array([cnt[(i - LEG_LENGTH) % n_pixels][0] for i in range(n_pixels)])
        x1s, y1s = point_1s[:, 0], point_1s[:, 1]
        x2s, y2s = point_2s[:, 0], point_2s[:, 1]
        
        phi_1s = np.degrees(np.arctan2(y1s - ys, x1s - xs) + np.pi)
        phi_2s = np.degrees(np.arctan2(y2s - ys, x2s - xs) + np.pi)
        
        indices = np.where(phi_2s > phi_1s)[0]
        
        for i in indices:
            phi1 = int(phi_1s[i] // BIN_SIZE) % N_ANGLE_BINS
            phi2 = int(phi_2s[i] // BIN_SIZE) % N_ANGLE_BINS
            hist[phi1, phi2] += 1
            
    normalised_hist = hist / np.sum(hist)
    feature_vector = normalised_hist[np.triu_indices_from(normalised_hist, k = 1)]
    
    return feature_vector

In [24]:
#Initialization:
N_RHO_BINS = 7
N_ANGLE_BINS = 12
N_BINS = N_RHO_BINS * N_ANGLE_BINS
BIN_SIZE = 360 // N_ANGLE_BINS
R_INNER = 5.0
R_OUTER = 35.0
K_S = np.arange(3, 8)
def get_cold_features(bw_image,contours, approx_poly_factor = 0.01 , sharpness_factor = 10 , bordersize = 3):

    #bw_image, _ = preprocess_image(img_file, sharpness_factor, bordersize)
    #contours = get_contour_pixels(bw_image)
    
    rho_bins_edges = np.log10(np.linspace(R_INNER, R_OUTER, N_RHO_BINS))
    feature_vectors = np.zeros((len(K_S), N_BINS))
    
    for j, k in enumerate(K_S):
        hist = np.zeros((N_RHO_BINS, N_ANGLE_BINS))
        for cnt in contours:
            epsilon = approx_poly_factor * cv2.arcLength(cnt,True)
            cnt = cv2.approxPolyDP(cnt,epsilon,True)
            n_pixels = len(cnt)
            
            point_1s = np.array([point[0] for point in cnt])
            x1s, y1s = point_1s[:, 0], point_1s[:, 1]
            point_2s = np.array([cnt[(i + k) % n_pixels][0] for i in range(n_pixels)])
            x2s, y2s = point_2s[:, 0], point_2s[:, 1]
            
            thetas = np.degrees(np.arctan2(y2s - y1s, x2s - x1s) + np.pi)
            rhos = np.sqrt((y2s - y1s) ** 2 + (x2s - x1s) ** 2)
            if(rhos.all()):
                rhos_log_space = np.log10(rhos)
            else:
                rhos_log_space = rhos
            
            quantized_rhos = np.zeros(rhos.shape, dtype=int)
            for i in range(N_RHO_BINS):
                quantized_rhos += (rhos_log_space < rho_bins_edges[i])
                
            for i, r_bin in enumerate(quantized_rhos):
                theta_bin = int(thetas[i] // BIN_SIZE) % N_ANGLE_BINS
                hist[r_bin - 1, theta_bin] += 1
            
        normalised_hist = hist / hist.sum()
        feature_vectors[j] = normalised_hist.flatten()
        
    return feature_vectors.flatten()

    

In [25]:
male_feature_cold=[]
female_feature_cold=[]
male_feature_hinge=[]
female_feature_hinge=[]

for i,img in enumerate(male_data):
    try:
        contours = get_contour_pixels(img)
        male_feature_cold.append(get_cold_features(img,contours))
        male_feature_hinge.append(get_hinge_features(img,contours))
    except :
        print(i)

for i,img in enumerate(female_data):
    try:
        contours = get_contour_pixels(img)
        female_feature_cold.append(get_cold_features(img,contours))
        female_feature_hinge.append(get_hinge_features(img,contours))
    except:
        print(i)

male_feature_cold=np.array(male_feature_cold)
female_feature_cold=np.array(female_feature_cold)
male_feature_hinge=np.array(male_feature_hinge)
female_feature_hinge=np.array(female_feature_hinge)

In [26]:

X_cold=np.concatenate((male_feature_cold,female_feature_cold),axis=0)

X_hinge=np.concatenate((male_feature_hinge,female_feature_hinge),axis=0)

Y=np.concatenate((np.ones(male_feature_cold.shape[0]),
                np.zeros(female_feature_cold.shape[0])),
                axis=0)

In [27]:
pca=PCA(n_components=50)
X_cold=pca.fit_transform(X_cold)
print(X_cold.shape, Y.shape)


(363, 50) (363,)


In [28]:
X=np.concatenate((X_cold,X_hinge),axis=1)
X=X_cold
X,Y=sklearn.utils.shuffle(X,Y)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=109) # 70% training and 30% test

In [29]:
estimator = []
estimator.append(('SVM', CalibratedClassifierCV(svm.SVC(kernel='rbf', verbose=True, C=10,gamma='auto'))))
#estimator.append(('RandomForst',
#                  RandomForestClassifier(n_estimators = 500, max_features = 'sqrt', criterion = 'gini', n_jobs=-1)))
#estimator.append(('NN',
#                 MLPClassifier(solver='lbfgs',         
#                    alpha=1e-5,
#                    hidden_layer_sizes=(2,), 
#                    random_state=1)))

In [33]:
vot_cold = VotingClassifier(estimators = estimator, voting ='soft')
vot_cold.fit(X_train, y_train)
y_pred=vot_cold.predict(X_test)
print(y_pred)
print("acc",metrics.accuracy_score(y_test,y_pred))
# cold_y_pred = vot_cold.predict(X_test[:,:50])
# print(cold_y_pred)
# cold_prob = vot_cold.predict_proba(X_test[:,:50])
# #print(lpb_prob)
# print("Accuracy:",metrics.accuracy_score(y_test, cold_y_pred))

[LibSVM]*
optimization finished, #iter = 100
obj = -1738.559940, rho = 0.996801
nSV = 178, nBSV = 171
Total nSV = 178
[LibSVM]*
optimization finished, #iter = 122
obj = -1738.938155, rho = 1.018195
nSV = 180, nBSV = 168
Total nSV = 180
[LibSVM]*
optimization finished, #iter = 115
obj = -1738.393592, rho = 1.003800
nSV = 178, nBSV = 170
Total nSV = 178
[LibSVM]*
optimization finished, #iter = 107
obj = -1738.238874, rho = 0.988976
nSV = 178, nBSV = 171
Total nSV = 178
[LibSVM]*
optimization finished, #iter = 114
obj = -1758.207663, rho = 1.010984
nSV = 182, nBSV = 171
Total nSV = 182
[1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1.]
acc 0.6712328767123288


In [41]:
vot_hinge = VotingClassifier(estimators = estimator, voting ='soft')
vot_hinge.fit(X_train[:,50:], y_train)

hinge_y_pred = vot_hinge.predict(X_test[:,50:])
print(hinge_y_pred)
hinge_prob = vot_hinge.predict_proba(X_test[:,50:])
#print(lpb_prob)
print("Accuracy:",metrics.accuracy_score(y_test, hinge_y_pred))

[LibSVM][LibSVM][LibSVM][1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
Accuracy: 0.6164383561643836


C:\Users\Raghod\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Raghod\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
C:\Users\Raghod\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:442: DeprecationWarning: `np.int` is a depre

In [29]:
clf = svm.SVC(kernel='rbf', verbose=True, C=10)

clf.fit(X_train, y_train)


[LibSVM]..*.

SVC(C=10, verbose=True)

*
optimization finished, #iter = 978
obj = -345.026860, rho = 0.688906
nSV = 182, nBSV = 11
Total nSV = 182


In [30]:

y_pred = clf.predict(X_test)

In [31]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.863013698630137


In [35]:
accuracy=0
for i in range(0,100):

    X=np.concatenate((male_feature,female_feature),axis=0)

    Y=np.concatenate((np.ones(male_feature.shape[0]),
                    np.zeros(female_feature.shape[0])),
                    axis=0)
    pca=PCA(n_components=50)
    X=pca.fit_transform(X)
    X,Y=sklearn.utils.shuffle(X,Y)

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=109) # 70% training and 30% test
    clf = svm.SVC(kernel='rbf', verbose=True, C=10)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    accuracy+=metrics.accuracy_score(y_test, y_pred)

    
    pass
print(accuracy/100)

[LibSVM]..*.*
optimization finished, #iter = 922
obj = -362.770486, rho = 0.629075
nSV = 180, nBSV = 12
Total nSV = 180
[LibSVM]..*.*
optimization finished, #iter = 890
obj = -375.294851, rho = 0.401559
nSV = 187, nBSV = 14
Total nSV = 187
[LibSVM]..*.*
optimization finished, #iter = 931
obj = -361.793426, rho = 0.389271
nSV = 185, nBSV = 11
Total nSV = 185
[LibSVM]..*.*
optimization finished, #iter = 1043
obj = -364.676491, rho = 0.795957
nSV = 184, nBSV = 9
Total nSV = 184
[LibSVM]..*.*
optimization finished, #iter = 924
obj = -325.721363, rho = 0.718731
nSV = 179, nBSV = 9
Total nSV = 179
[LibSVM]..*.*
optimization finished, #iter = 983
obj = -340.634794, rho = 0.697520
nSV = 185, nBSV = 11
Total nSV = 185
[LibSVM]..*.*
optimization finished, #iter = 940
obj = -356.291803, rho = 0.606962
nSV = 185, nBSV = 11
Total nSV = 185
[LibSVM]..*.*
optimization finished, #iter = 956
obj = -369.605406, rho = 0.483333
nSV = 186, nBSV = 12
Total nSV = 186
[LibSVM]..*
optimization finished, #iter 

In [ ]:

a=np.ones((3,3))
b=np.zeros((3,3))
print(np.concatenate(  (a,b),axis=1)  )